In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
# Weight Decision - 熵值法
def entropy(x):
    rows = x.index.size
    cols = x.columns.size
    k = 1.0 / math.log(rows)
 
    lnf = [[None] * cols for i in range(rows)]
    x = np.array(x)
    lnf = [[None] * cols for i in range(rows)]
    lnf = np.array(lnf)
    for i in range(0, rows):
        for j in range(0, cols):
            if x[i][j] == 0:
                lnfij = 0.0
            else:
                p = x[i][j] / x.sum(axis=0)[j]
                lnfij = math.log(p) * p * (-k)
            lnf[i][j] = lnfij
    lnf = pd.DataFrame(lnf)
    E = lnf
 

    d = 1 - E.sum(axis=0)
    w = [[None] * 1 for i in range(cols)]
    for j in range(0, cols):
        wj = d[j] / sum(d)
        w[j] = wj
    
    w = pd.DataFrame(w)
    return w

In [3]:
# Normalization Method
def Linear_Max_Min(attribute, Type):
    """
    Patro, S. and Sahu, K.K., 2015. Normalization: A preprocessing stage.
    arXiv preprint arXiv:1503.06462.
    """
    MIN = min(attribute)
    MAX = max(attribute)
    Var = MAX - MIN
    attribute = np.array(attribute)
    if Type == "profit":
        Attribute = (attribute - MIN)/Var
    elif Type == "cost":
        Attribute = (MAX - attribute)/Var
    
    return Attribute

def Vector_Normalization(attribute, Type):
    """
    Jahan, A. and Edwards, K.L. (2014) ‘A state-of-the-art survey on the
    influence of normalization techniques in ranking: improving the materials
    selection process in engineering design’, Mater. Des., Vol. 65, No. 2015,
    pp.335–342.
    """
    r = (sum(attribute**2))**(0.5)
    if Type == "profit":
        Attribute = attribute/r 
    elif Type == "cost":
        Attribute = 1 - attribute/r 
        
    return Attribute

# To Normalize the Decision Matrix
def Normalize_Decision_Matrix(Decision_Matrix, Attribute_Type):
    for i in range(0, len(Attribute_Type)):
        if Attribute_Type[i] == 1:
            Type = "profit"
        elif Attribute_Type[i] == 0:
            Type = "cost"
        Decision_Matrix.iloc[:, i] = Vector_Normalization(Decision_Matrix.iloc[:, i], Type)
    return Decision_Matrix

In [4]:
# Find Ideal Solutions
def Ideal_Solution(Weighted_normalized_Decision_matrix, Attribute_Type):
    Positive_Ideal_Solutions = []
    Negative_Ideal_Solutions = []
    for i in range(0, len(Attribute_Type)):
        if Attribute_Type[i] == 1:
            Positive_Ideal = max(Weighted_normalized_Decision_matrix.iloc[:, i]) 
            Negative_Ideal = min(Weighted_normalized_Decision_matrix.iloc[:, i])
            
            Positive_Ideal_Solutions.append(Positive_Ideal)
            Negative_Ideal_Solutions.append(Negative_Ideal)
        elif Attribute_Type[i] == 0:
            Positive_Ideal = min(Weighted_normalized_Decision_matrix.iloc[:, i]) 
            Negative_Ideal = max(Weighted_normalized_Decision_matrix.iloc[:, i])
        
            Positive_Ideal_Solutions.append(Positive_Ideal)
            Negative_Ideal_Solutions.append(Negative_Ideal)
            
    return (Positive_Ideal_Solutions, Negative_Ideal_Solutions)

# Find Distance
def Euclidean_Distance(Ideal_Points, Weighted_normalizaed_Decision_matrix, Attribute_Type):
    Positive_Ideal_Point = Ideal_Solution(Weighted_normalizaed_Decision_matrix, Attribute_Type)[0]
    Negative_Ideal_Point = Ideal_Solution(Weighted_normalizaed_Decision_matrix, Attribute_Type)[1]
    
    Negative_Si = (np.sum((Weighted_normalizaed_Decision_matrix-Negative_Ideal_Point)**2, axis = 1))**0.5  
    Positive_Si = np.sum((Weighted_normalizaed_Decision_matrix-Positive_Ideal_Point)**2, axis = 1)**0.5

    C = Negative_Si/(Negative_Si + Positive_Si)
    return (Weighted_normalizaed_Decision_matrix,C)

In [5]:
# Weighted Matrix
def Weight_Prod(Decision_Matrix, weights):
    if sum(weights) != 1:
        print("Shape of weights is not satisfied.weights must be equal 1")
    else:
        if Decision_Matrix.shape[1] != len(weights):
            print("Shape of weights is not satisfied.")
        else:
            for i in range (0, Decision_Matrix.shape[1]):
                Decision_Matrix.iloc[:, i] = Decision_Matrix.iloc[:, i]*weights[i]
                
        return Decision_Matrix

In [6]:
def TOPSIS(Normalize_Decision_Matrix, Attribute_Types, Weights):
    Weighted_Normalizaed_Decision_matrix = Weight_Prod(Decision_Matrix = Normalize_Decision_Matrix, weights = Weights)
    Ideal_Solutions = Ideal_Solution(Weighted_Normalizaed_Decision_matrix, Attribute_Types)
    a = Euclidean_Distance(Ideal_Solutions, Weighted_Normalizaed_Decision_matrix, Attribute_Types)
    return a

In [10]:
attributes = np.array(["Noise Pollution","CO2 Emission of Driving", "Energy Consumption","CO2 Emission of Production"])
candidates = np.array(["Electric","BioDiesel", "BioGas", "Diesel"])
raw_data = np.array([
    [63,0,1.5,3.3183],
    [68,12.76,4.5,3.3993],
    [70,13.55,6,2.5349],
    [68, 1151.4, 4.13, 8.88]
])
benefit_attributes = [0,0,0,0]
Data_Ori = pd.DataFrame(data=raw_data, index=candidates, columns=attributes)

Normalized_Data = Normalize_Decision_Matrix(Decision_Matrix = Data_Ori,Attribute_Type=benefit_attributes)
display(Normalized_Data)

Weights = entropy(Normalized_Data)

Weights.index = Normalized_Data.columns
Weights.columns = ['weight']
display(Weights)

wgt = Weights['weight'].tolist()

Output = TOPSIS(Normalized_Data,Attribute_Types=[1,1,1,1],Weights=wgt)
Scores=pd.DataFrame(Output[1])
Normalized_Weighted_Decision_Matrix = Output[0]
df=pd.concat([Normalized_Weighted_Decision_Matrix, Scores], axis=1)
display(df)

,Noise Pollution,CO2 Emission of Driving,Energy Consumption,CO2 Emission of Production
Electric,0.531944,1.000000,0.827434,0.680469
BioDiesel,0.494797,0.988919,0.482303,0.672669
BioGas,0.479938,0.988233,0.309738,0.755906
Diesel,0.494797,0.000131,0.524869,0.144914


,weight
Noise Pollution,0.001559
CO2 Emission of Driving,0.613422
Energy Consumption,0.126605
CO2 Emission of Production,0.258414


,Noise Pollution,CO2 Emission of Driving,Energy Consumption,CO2 Emission of Production,0
Electric,0.000829,0.613422,0.104757,0.175843,0.970086
BioDiesel,0.000771,0.606625,0.061062,0.173827,0.926741
BioGas,0.000748,0.606204,0.039214,0.195337,0.904752
Diesel,0.000771,0.000080,0.066451,0.037448,0.041160
